<a href="https://colab.research.google.com/github/josefl19/CustomVision_clasificador/blob/main/retoSemana3_ClasificadorImagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificador de Imagenes

Texto para después.

In [1]:
pip install azure-cognitiveservices-vision-customvision

     |████████████████████████████████| 71kB 738kB/s 
     |████████████████████████████████| 92kB 1.8MB/s 
     |████████████████████████████████| 51kB 1.4MB/s 


In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import time

In [3]:
# Replace with valid values
ENDPOINT = "https://clasificador-imagenes.cognitiveservices.azure.com/"
training_key = "8d48037707544f4e8f853a11878aa59d"
prediction_key = "3b6dd54ac1054f2883f9d121c4a1191d"
prediction_resource_id = "d7b2c753-e443-4211-9592-14e43d67e24b"

In [4]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [5]:
publish_iteration_name = "clasificadorMosca"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Create a new project
print ("Creating project...")
project = trainer.create_project("Proyecto de Clasificación con Custom Vision")

Creating project...


In [6]:
# Make two tags in the new project
mosca_tag = trainer.create_tag(project.id, "Mosca")
gusano_tag = trainer.create_tag(project.id, "Gusano")
palomilla_tag = trainer.create_tag(project.id, "Palomilla")
pulgon_tag = trainer.create_tag(project.id, "Pulgon")

In [9]:
base_image_location = "https://github.com/josefl19/CustomVision_clasificador/tree/main/"
print("Adding images...")

image_list = []

# Lectura de las imagenes de mosca blanca.
for image_num in range(1, 30):
    file_name = "mosca{}.jpg".format(image_num)
    with open(base_image_location + "img_etiquetas/mosca/" + file_name, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[mosca_tag.id]))

# Lectura de las imagenes de gusanos.
for image_num in range(1, 12):
    file_name = "gusano_{}.jpg".format(image_num)
    with open(base_image_location + "img_etiquetas/gusano/" + file_name, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[gusano_tag.id]))

# Lectura de las imagenes de palomillas.
for image_num in range(1, 12):
    file_name = "palomilla_{}.jpg".format(image_num)
    with open(base_image_location + "img_etiquetas/palomilla/" + file_name, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[palomilla_tag.id]))

# Lectura de las imagenes del pulgones.
for image_num in range(1, 17):
    file_name = "pulgon_{}.jpg".format(image_num)
    with open(base_image_location + "img_etiquetas/pulgon/" + file_name, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[pulgon_tag.id]))

upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

Adding images...


FileNotFoundError: ignored

In [ ]:
print ("Entrenando modelo...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

In [ ]:
# Now there is a trained endpoint that can be used to make a prediction
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

with open(base_image_location + "img_test/test_1.jpg", "rb") as image_contents:
    results = predictor.classify_image(
        project.id, publish_iteration_name, image_contents.read())

    # Display the results.
    for prediction in results.predictions:
        print("\t" + prediction.tag_name +
              ": {0:.2f}%".format(prediction.probability * 100))